# kaggle - Learn: Intermediate Machine Learning
- https://www.kaggle.com/learn/intermediate-machine-learning

## 2.- Missing Values
- Missing values happen. Be prepared for this common challenge in real datasets.
- Most machine learning libraries (including scikit-learn) give an error if you try to build a model using data with missing values.
-  you will learn three approaches to __dealing with missing values__. Then you'll compare the effectiveness of these approaches on a real-world dataset.
    1. A Simple Option: Drop Columns with Missing Values
    2. A Better Option: Imputation - fill missing value with some number (ex. mean())
    3. An Extension To Imputation - 2. + new bool missing column

## A Case
- In the example, we will work with min_melb_data.csv dataset in files/ subdir. Our model will use information such as the number of rooms and land size to predict home price.

In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('files/min_melb_data.csv')     # Load the data
print(df.shape)     #display(df)
y = df.Price                                    # Select target

# To keep things simple, we'll use only numerical predictors (features). NEW!
melb_predictors = df.drop(['Price'], axis=1)            # new df w/o Price col
X = melb_predictors.select_dtypes(exclude=['object'])   # new df with only numerical cols

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=0,
                                                        train_size=0.8,
                                                        test_size=0.2)

### JM TOUCH cause ALL the cols have missing values, look
## df.info() mmm¿?
#X_valid.info()
#X_train.info()
#display(X_train[pd.isnull(X_train.Distance)])  # row[1920]
X_train.drop([1920], inplace=True)
#display(X_train[pd.isnull(X_train.Distance)])  # row[1920]
#X_train.info()
## Now we have not all cols with NaN.. --> in X_train!!
#display(X_valid[pd.isnull(X_valid.Distance)])   # row[2130]
X_valid.drop([2130], inplace=True)
#display(X_valid[pd.isnull(X_valid.Distance)])   # row[2130]
## Now ..ok!!



(2679, 21)


### Def. Funct. to Measure Quality of Each Approach to dealing w/missing values
We define a function score_dataset() to compare different approaches to dealing with missing values. This function reports the mean absolute error (MAE) from a random forest model.


In [46]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_t, X_v, y_t, y_v):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_t, y_t)
    y_pred = model.predict(X_v)
    return mean_absolute_error(y_v, y_pred)

### Score from Approach 1 (Drop cols with missing values)
Since we are working with both training and validation sets, we are careful to drop the same columns in both DataFrames.
> JM: dorp missing-values cols BEFORE train_test_split? - future...

In [47]:
# X_train.columns, type(X_train.columns)
# ['Rooms', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea',
# 'YearBuilt', 'Lattitude', 'Longtitude', 'Propertycount'] - pandas.core.indexes.base.Index

cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]

# Drops missing_vals_cols in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE from Approach 1 (Drop columns with missing values):")
# Error!! - print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))
# ValueError: Input y contains NaN.

## Eliminate NaN in y_train and y_valid
# print(y_train[pd.isnull(y_train)])
# print(y_valid[pd.isnull(y_valid)])
y_train.dropna(inplace=True)
y_valid.dropna(inplace=True)
# print(y_valid[pd.isnull(y_valid)])

print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop columns with missing values):
217286.62174454826


### Score from Approach 2 (Imputation)
We'll use *SimpleImputer* to replace missing values with the mean value along each column
- filling in the mean value generally performs quiet well, but this varies by dataset.
- Statisticians have experimented with more complex ways to determine imputed values (such as regression imputation), but typically give no additional benefit.

In [48]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from Approach 2 (Imputation):
210048.71943925234


### Score from Approach 3 (An Extension to Imputation)
Next, we impute the missing values, while also keeping track of which values were imputed.

In [49]:
# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

MAE from Approach 3 (An Extension to Imputation):
205376.4110280374


As we can see, Approach 3 performed slightly worse than Approach 2.    
__So, why did imputation perform better than dropping the columns?__    
The training data has 10864 rows and 12 columns (JM: df.shape = (2679, 21)), where three (JM: many) columns contain missing data. For each column, less than half of the entries are missing. Thus, dropping the columns removes a lot of useful information, and so it makes sense that imputation would perform better.

In [50]:
print(X_train.shape)
# Number of missing values in each column of training data
missing_val_count_by_column = X_train.isnull().sum()
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(2142, 12)
Car              16
BuildingArea    943
YearBuilt       788
dtype: int64


### Conclusion
As is common, imputing missing values (in Approach 2 and Approach 3) yielded better results, relative to when we simply dropped columns with missing values (in Approach 1).

# FUTURE more análysis ... 
## JM try to fill with mean - similar approach 2
- BUT using fillna an col.mean() val

### First evaluate (see) NaNs in X_train, X_valid, y_train, y_valid

In [51]:
def show_nullsum(pdo_name, pdo):
    missings_by_col = X_train.isnull().sum()
    print(pdo_name, '  - shape:', pdo.shape, '  - type:', type(pdo))
    print(missings_by_col[missings_by_col > 0])
    print()

d = {'X_train': X_train, 'X_valid': X_valid, 'y_train': y_train, 'y_valid': y_valid }
for k,v in d.items():
    show_nullsum(k, v)

# X_train.isnull().sum()
# X_valid.isnull().sum()
# y_train.isnull().sum()
# y_valid.isnull().sum()
# X_train.__dict__

X_train   - shape: (2142, 12)   - type: <class 'pandas.core.frame.DataFrame'>
Car              16
BuildingArea    943
YearBuilt       788
dtype: int64

X_valid   - shape: (535, 12)   - type: <class 'pandas.core.frame.DataFrame'>
Car              16
BuildingArea    943
YearBuilt       788
dtype: int64

y_train   - shape: (2142,)   - type: <class 'pandas.core.series.Series'>
Car              16
BuildingArea    943
YearBuilt       788
dtype: int64

y_valid   - shape: (535,)   - type: <class 'pandas.core.series.Series'>
Car              16
BuildingArea    943
YearBuilt       788
dtype: int64



In [55]:
## Filling all the values with the mean() of corresponding columnn
missing_vals_cols = ['Car', 'BuildingArea', 'YearBuilt']

for col in missing_vals_cols:
    X_train[col] = X_train[col].fillna(X_train[col].mean())
    X_valid[col] = X_valid[col].fillna(X_valid[col].mean())

# # for pdo in [X_train, y_train]:
# #     for col in missing_vals_cols:
# #         pdo[col] = pdo[col].fillna(pdo[col].mean())

# for col in missing_vals_cols:
#     X_valid[col] = X_valid[col].fillna(pdo[col].mean())

show_nullsum('X_train', X_train)
show_nullsum('y_train', y_train)


X_train   - shape: (2142, 12)   - type: <class 'pandas.core.frame.DataFrame'>
Series([], dtype: int64)

y_train   - shape: (2142,)   - type: <class 'pandas.core.series.Series'>
Series([], dtype: int64)



In [53]:
display(X_valid[pd.isnull(X_valid['Car'])])
display(X_valid)

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount


,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
1509,2.0,13.6,3148.0,2.0,1.0,1.0,842.0,150.065789,1972.000000,-37.88590,145.09790,3582.0
2603,3.0,14.2,3149.0,3.0,2.0,2.0,236.0,138.000000,2006.000000,-37.88678,145.12748,13366.0
2229,3.0,5.2,3055.0,3.0,1.0,2.0,399.0,105.000000,1950.000000,-37.75744,144.94945,7082.0
92,3.0,3.3,3206.0,3.0,2.0,1.0,177.0,181.000000,1880.000000,-37.84150,144.95850,3280.0
1773,4.0,13.9,3108.0,4.0,2.0,2.0,716.0,155.000000,1965.000000,-37.77800,145.11730,9028.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1225,3.0,7.8,3124.0,3.0,2.0,2.0,656.0,225.000000,2009.000000,-37.84100,145.08240,8920.0
1748,3.0,1.6,3066.0,2.0,2.0,2.0,0.0,159.000000,1920.000000,-37.80420,144.98450,4553.0
2297,4.0,13.8,3188.0,4.0,1.0,1.0,700.0,175.000000,1950.000000,-37.94312,145.02440,2356.0
2342,3.0,18.8,3170.0,3.0,1.0,2.0,656.0,107.000000,1972.000000,-37.93271,145.17792,7113.0
